In [245]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
#pd.read_html('https://fantasy.espn.com/tournament-challenge-bracket/2022/en/whopickedwhom')[0]
response = requests.get('https://projects.fivethirtyeight.com/march-madness-api/2022/madness.json')
# soup = BeautifulSoup(page.content, 'html.parser')
# soup

json_data = response.json()
json_data['games']
forecast_data = json_data['forecasts']['mens']['current_run']['teams']#[0]
df = pd.DataFrame(forecast_data)
df = df[['team_name', 'team_rating', 'team_region', 'team_seed', 'rd2_win', 'rd3_win', 'rd4_win', 'rd5_win', 'rd6_win', 'rd7_win']]
df.columns = ['name', 'rating', 'region', 'seed', '32', '16', '8', '4', '2', '1']

#print(df.head())
def combine_teams(name1, name2, combined_name, df):
    team1 = df.loc[df['name'] == name1]
    team2 = df.loc[df['name'] == name2]
    #print(team1)
    #print(team2)
    row_combined = []
    for i in range(10):
        if i == 0:
            row_combined.append(combined_name)
        elif i ==1:
            row_combined.append((team1.iloc[0,i]+team2.iloc[0,i])/2)
        elif i == 2:
             row_combined.append(team1.iloc[0,i])
        elif i == 3:
             row_combined.append(team1.iloc[0,i][0:2])
        else:
            row_combined.append(team1.iloc[0,i]+team2.iloc[0,i])
            

    df = df[((df.name != name1) & (df.name != name2))]
    row_combined_series = pd.Series(row_combined, index=df.columns)
    df = df.append(row_combined_series, ignore_index=True)
    return df

# df = combine_teams('Wyoming', 'Indiana', 'WYO/IU', df)
# df = combine_teams('Wright State', 'Bryant', 'WRST/BRY', df)
# df = combine_teams('Rutgers', 'Notre Dame', 'RUTG/ND', df)
# df = combine_teams('Texas Southern', 'Texas A&M-Corpus Christi', 'TXSO/TCC', df)

# WYO/IU
# RUTG/ND
# WRST/BRY
# TXSO/TCC

def new_name(name, new_name, df):
    df.loc[df.name == name, 'name'] = new_name#.tolist()
    return df
             
df = new_name('Louisiana State', 'LSU', df)
df = new_name('Loyola (IL)', 'Loyola Chicago', df)
df = new_name('Saint Mary\'s (CA)', 'Saint Mary\'s', df)
df = new_name('Connecticut', 'UConn', df)
df = new_name('Alabama-Birmingham', 'UAB', df)
df = new_name('Texas Christian', 'TCU', df)
df = new_name('Miami (FL)', 'Miami', df)
df = new_name( 'Jacksonville State', 'J\'Ville St', df)
df = new_name('Cal State Fullerton','CSU Fullerton', df)
df = new_name( 'Southern California', 'USC', df)
df = new_name('South Dakota State','S Dakota St', df)
df = new_name('New Mexico State', 'New Mexico St', df)
df = new_name('Saint Mary\'s (CA)', 'Saint Mary\'s', df)



In [246]:
ret = []
starting_round = 0
for index, row in df.iterrows():
    if row[6] == 0:
        df = df.loc[df.name != row[0]]
        continue
#     print(index)
#     print(row[4])
#     print(type(row[4]))
#     print(row[4]==0)
    ret.append({'name': row[0], 'round_64-32': row[4], 'round_32-16': row[5]/row[4], 'round_16-8': row[6]/row[5], 
                'round_8-4': row[7]/row[6], 'round_4-2': row[8]/row[7], 
                'round_2-1': row[9]/row[8]})

df_round_odds_given_previous_wins = pd.DataFrame(ret)
df_round_odds_given_previous_wins = df_round_odds_given_previous_wins[['name', 'round_64-32', 'round_32-16', 'round_16-8', 'round_8-4', 'round_4-2', 'round_2-1']]
df_round_odds_given_previous_wins

optimal_round = []
for index, row in df_round_odds_given_previous_wins.iterrows():
    max_diff = 0
    for i in range(2,len(row)):
        temp = row[i-1]-row[i]
        if temp > max_diff:
            max_diff = temp
            max_diff_index = i
    
    optimal_round.append(df_round_odds_given_previous_wins.columns[max_diff_index-1])
    
df_round_odds_given_previous_wins['optimal_round'] = optimal_round

df_round_odds_given_previous_wins.head()

for index, line in df_round_odds_given_previous_wins.iterrows():
    print(line[0], ' - ', line[7], '\t\t\t', line[1])



Gonzaga  -  round_8-4 			 0.988339637138
Kansas  -  round_32-16 			 0.968928817195
Kentucky  -  round_32-16 			 0.95486796975
Arizona  -  round_32-16 			 0.963036481526
Auburn  -  round_16-8 			 0.945098252345
Villanova  -  round_64-32 			 0.948320571996
Iowa  -  round_32-16 			 0.816344258688
Houston  -  round_64-32 			 0.768048297263
Purdue  -  round_64-32 			 0.939351258825
Tennessee  -  round_64-32 			 0.941402378758
UCLA  -  round_64-32 			 0.925615737287
Duke  -  round_64-32 			 0.970644764402
Texas Tech  -  round_16-8 			 0.926978076152
Baylor  -  round_64-32 			 0.963014020329
Illinois  -  round_64-32 			 0.806639407015
LSU  -  round_64-32 			 0.671008513676
Arkansas  -  round_32-16 			 0.717856124793
Wisconsin  -  round_64-32 			 0.848889049507
Texas  -  round_64-32 			 0.541929499529
UConn  -  round_32-16 			 0.862753431513
Alabama  -  round_64-32 			 0.63251135527
Michigan  -  round_64-32 			 0.689911398471
Virginia Tech  -  round_8-4 			 0.458070500471
North Carolina  -  ro

In [247]:
import re
###### taking pick data from ESPN now
df_picks = pd.read_html('https://fantasy.espn.com/tournament-challenge-bracket/2022/en/whopickedwhom')[0]
#df_picks_fixed = []

picks_dict = {}
for index, row in df_picks.iterrows():
    
    #df_picks_fixed.append([name])
    #print(row)
    for i in range(len(row)):
        name = re.search('\d*([a-zA-Z\s\'\./]*)-.*', row[i]).group(1)
        picks_dict.setdefault(name, [None,None,None,None,None,None])
        
        percentage = re.search('\d+\.\d', row[i]).group(0)
        picks_dict[name][i] = float(percentage)
        
        #df_picks_fixed[index].append(float(percentage))
    
df_picks = pd.DataFrame.from_dict(picks_dict, orient='index', columns = [ '32', '16', '8', '4', '2', '1'])
df_picks.head()

#check if name in dict
#if not in add name with empty list 6 items long
#insert into dict key in ith place the number found

,32,16,8,4,2,1
Arizona,96.8,88.9,67.2,43.3,29.8,13.4
Gonzaga,96.5,90.3,78.7,58.4,41.7,27.1
Kansas,96.6,87.4,64.9,42.7,19.3,8.3
Baylor,96.2,78.6,53.7,28.5,10.5,5.4
Kentucky,95.6,83.2,54.6,31.9,13.0,6.9


In [248]:
ESPN_teams = df_picks.index.unique().tolist()
fivethirtyeight_teams = df.name.unique().tolist()
for name in ESPN_teams:
    if name not in fivethirtyeight_teams:
        print(name)

print('\n----------------------\n')
for name in fivethirtyeight_teams:
    if name not in ESPN_teams:
        print(name)


----------------------



In [249]:
### df is fivethirty eight data on how likely a team is to make it to each round
# this will be used with public information about how far each team is picked to determine what picks
# offer the most leverage
column_choices = [0,4,5,6,7,8,9]
probabilities = df[df.columns[column_choices]]
probabilities = probabilities.sort_values(by=['name']).reset_index(drop = True)

df_picks.reset_index(inplace=True)
df_picks_index_drop = df_picks.sort_values(by=['index']).reset_index(drop = True)
print(df_picks_index_drop.head())


leverage_table = []

for index, row in df_picks_index_drop.iterrows():
    leverage_row = [probabilities.name[index]]

    for i in range(1,len(row)):
        #accoutn for different point values in different rounds
        #this calc works correctly
        if i == 1:
            leverage_row.append((probabilities.iloc[index][i] - df_picks_index_drop.iloc[index][i]/100)*2**(i-1))
            
        #account for previous rounds as well
        # a team may be under picked for the elite eight but over picked for the sweet sixteen
        #removing some of the value that they would otherwise bring
        #for example in 2019 LSU was overpicked to win the first game but under picked to win more
        # they provided negative leverage at first but this was over powered by the leverage brought later
        # but the negative leverage from previous rounds must still be accounted for.
        elif i > 1:
            #account for most recent round as well (which in turn also counts previous rounds)
            #this does not give a leverage score of the pure round pick but the whole path to this point for the team
            leverage_row.append((probabilities.iloc[index][i] - df_picks_index_drop.iloc[index][i]/100)*2**(i-1) + leverage_row[len(leverage_row)-1])
    leverage_table.append(leverage_row)
        
leverage_table = pd.DataFrame(leverage_table, columns = ['name', '32', '16', '8', '4', '2', '1']) 
print(leverage_table.head())
leverage_table.to_csv('leverage_picks.csv')


      index    32    16     8     4     2     1
0     Akron   9.1   3.2   0.8   0.3   0.1   0.1
1   Alabama  71.1  21.7   7.2   1.8   0.7   0.4
2   Arizona  96.8  88.9  67.2  43.3  29.8  13.4
3  Arkansas  76.8  46.4   8.4   3.7   1.6   0.8
4    Auburn  95.0  78.1  52.5  23.7  10.4   4.3
       name        32        16         8         4         2         1
0     Akron -0.016616 -0.048168 -0.062811 -0.080164 -0.094204 -0.125341
1   Alabama -0.078489 -0.036656  0.111210  0.166103  0.243375  0.292922
2   Arizona -0.004964 -0.171510 -0.941929 -2.107372 -4.441478 -6.539499
3  Arkansas -0.050144 -0.213272 -0.229729 -0.191358 -0.115415 -0.047738
4    Auburn -0.004902 -0.050009 -0.207969 -0.270924  0.031748  0.472124


['Akron', -0.008527537347999999, -0.025582612043999996, -0.062848076608, -0.077809009252, -0.095399813932, -0.12551732025200002, -0.156700007612]
